# LLM tooling for Chemistry

## *Gemini*

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
June 2025

-----

## Introduction

This notebook shows how to do parallel [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) workflows with Large Language Models (LLMs) of Gemini. 

The Raku package ["WWW::Gemini"](https://github.com/antononcube/Raku-WWW-Gemini), [AAp2], is used.

### Examples and big picture

The rest of the notebook gives concrete code how to do streamline multiple-tool function calling with Gemini's LLMs using Raku. 
Gemini's function calling example ["Parallel Function Calling"](https://ai.google.dev/gemini-api/docs/function-calling#parallel_function_calling), [Gem1], is followed.

This notebook belongs to a collection of notebooks describing how to do LLM function calling with Raku.

Compared to the previously described LLM workflows with OpenAI, [[AA1](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/)], and Gemini, [[AA2](https://rakuforprediction.wordpress.com/2025/06/07/llm-function-calling-workflows-part-2-googles-gemini/)], the Gemini LLM workflow in this notebook demonstrates:

- Use of multiple tools (parallel function calling)
- Automatic generation of hashmap (or JSON) tool descriptors
- Streamlined computation of multiple tool results from multiple LLM requests

The streamlining is achieved by using the provided by ["LLM::Functions"](https://raku.land/zef:antononcube/LLM::Functions), [AAp3]:

- Classes `LLM::Tool`, `LLM::ToolRequest`, and `LLM::ToolResult`
- Subs `llm-tool-definition` and `generate-llm-tool-result`
    - The former sub leverages Raku's introspection features. 
    - The latter sub matches tools and requests in order to compute tool responses.

-----

## Setup

Load packages:

In [21]:
use JSON::Fast;
use Data::Reshapers;
use Data::TypeSystem;
use LLM::Tooling;
use LLM::Functions;
use LLM::Graph;
use Chemistry::Stoichiometry;
use WWW::Gemini;

Choose a model:

In [22]:
my $model = "gemini-2.0-flash";

gemini-2.0-flash

------

## Workflow

### Define a local function

Define a few subs -- _tools_ -- with sub- and argument descriptions (i.e. attached Pod values, or [declarator blocks](https://docs.raku.org/language/pod#Declarator_blocks)):

In [23]:
&balance-chemical-equation.WHY

Balance a chemical equation.

In [24]:
balance-chemical-equation('C2H5OH + O2 = H2O + CO2')

[1*C2H5OH + 3*O2 -> 2*CO2 + 3*H2O]

In [25]:
molecular-mass('15*SO2')

960.8700000000001

In [26]:
&molecular-mass.WHY

Convert chemical compound formula (molecule) into molecule mass. * C<$specs | @specs> A string, or a list of strings. * C<$:p> Should the result be a list of pairs or not? (If the first argument is positional, C<@specs>.)

In [27]:
&chemical-element-data.WHY

Convert chemical element names, abbreviations, or atomic numbers into chemical element data records. * C<$num> A string, an integer, or a list of strings and/or integers to be converted. * C<$symbol> Adverb to return symbol. * C<$abbr> Adverb to return symbol. (Same as C<$symbol>.) * C<$name> Adverb to return standard name. * C<$standard-name> Adverb to return standard name. (Same as C<$name>.) * C<$weight> Adverb to return atomic weight. * C<$atomic-weight> Adverb to return atomic weight. (Same as C<$weight>.) * C<$number> Adverb to return atomic number. * C<$atomic-number> Adverb to return atomic number. (Same as C<$number>.)

**Remark:** See the corresponding Python definitions in the section ["Parallel Function Calling"](https://ai.google.dev/gemini-api/docs/function-calling#parallel_function_calling) of [Gem1].

The sub `llm-tool-definition` can be used to _automatically_ generate the Raku-hashmaps or JSON-strings of the tool descriptors in the (somewhat universal) format required by LLMs:

In [28]:
llm-tool-definition(&chemical-element-data, format => 'json')

Undefined type of parameter ⎡$spec⎦; continue assuming it is a string.


{
  "function": {
    "description": "Convert chemical element names, abbreviations, or atomic numbers into chemical element data records. * C<$num> A string, an integer, or a list of strings and/or integers to be converted. * C<$symbol> Adverb to return symbol. * C<$abbr> Adverb to return symbol. (Same as C<$symbol>.) * C<$name> Adverb to return standard name. * C<$standard-name> Adverb to return standard name. (Same as C<$name>.) * C<$weight> Adverb to return atomic weight. * C<$atomic-weight> Adverb to return atomic weight. (Same as C<$weight>.) * C<$number> Adverb to return atomic number. * C<$atomic-number> Adverb to return atomic number. (Same as C<$number>.)",
    "strict": true,
    "parameters": {
      "type": "object",
      "required": [
        "$spec"
      ],
      "properties": {
        "$standard-name": {
          "type": "integer",
          "description": "return standard name"
        },
        "$atomic-number": {
          "type": "integer",
          "descripti

**Remark:** The sub `llm-tool-description` is invoked in `LLM::Tool.new`. Hence (ideally) `llm-tool-description` would not be user-invoked that often.

These are the tool descriptions to be communicated to Gemini:

In [33]:
llm-tool-definition(&molecular-mass)

Undefined type of parameter ⎡$spec⎦; continue assuming it is a string.


{
  "function": {
    "description": "Convert chemical compound formula (molecule) into molecule mass. * C<$specs | @specs> A string, or a list of strings. * C<$:p> Should the result be a list of pairs or not? (If the first argument is positional, C<@specs>.)",
    "strict": true,
    "type": "function",
    "name": "molecular-mass",
    "parameters": {
      "additionalProperties": false,
      "required": [
        "$spec"
      ],
      "properties": {
        "$spec": {
          "description": "A molecule formula, or a list of molecule formulas",
          "type": "string"
        }
      },
      "type": "object"
    }
  },
  "type": "function"
}

In [9]:
my @tools =
{
    :name("molecular-mass"), 
    :description("Convert chemical compound formula (molecule) into molecule mass."), 
    :parameters(
        {
            :type("object")
            :properties( {"\$spec" => {:description("A molecule formula, or a list of molecule formulas"), :type("string")}}), 
            :required(["\$spec"]), 
        }), 
},
{
    :name("balance-chemical-equation"), 
    :description("Balance the given chemical equation."), 
    :parameters(
        {
            :type("object")
            :properties( {"\$spec" => {:description("A chemical equation to be balanced"), :type("string")}}), 
            :required(["\$spec"]), 
        }), 
},
;

deduce-type(@tools)

Vector(Struct([description, name, parameters], [Str, Str, Hash]), 2)

Here are additional tool-mode configurations (see ["Function calling modes"](https://ai.google.dev/gemini-api/docs/function-calling?example=weather#function_calling_modes) of [Gem1]):

In [10]:
my %toolConfig =
  functionCallingConfig => {
    mode => "ANY",
    allowedFunctionNames => <molecular-mass balance-chemical-equation>
  };

{functionCallingConfig => {allowedFunctionNames => (molecular-mass balance-chemical-equation), mode => ANY}}

### First communication with Gemini

Initialize messages:

In [34]:
# User prompt
# my $prompt = q:to/END/;
# What is the mass of SO2 ?
# Also, can you balance: 
#     C2H5OH + O2 = 2*H2O + CO2
# END

my $prompt = q:to/END/;
What are the masses of SO2, 03, and C2H5OH?
END

# Prepare the API request payload
my @messages = [{role => 'user',parts => [ %( text => $prompt ) ]}, ];

[{parts => [text => What are the masses of SO2, 03, and C2H5OH?
], role => user}]

Send the first chat completion request:

In [35]:
my $response = gemini-generate-content(
    @messages,
    :$model,
    :@tools,
    :%toolConfig
);

deduce-type($response)

Struct([candidates, modelVersion, responseId, usageMetadata], [Hash, Str, Str, Hash])

In [36]:
deduce-type($response)

Struct([candidates, modelVersion, responseId, usageMetadata], [Hash, Str, Str, Hash])

The response is already parsed from JSON to Raku. Here is its JSON form:

In [37]:
to-json($response)

{
  "usageMetadata": {
    "candidatesTokenCount": 24,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 73
      }
    ],
    "promptTokenCount": 73,
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 24
      }
    ],
    "totalTokenCount": 97
  },
  "responseId": "GaW9aKKPDcCmmtkPvdKX-QI",
  "modelVersion": "gemini-2.0-flash",
  "candidates": [
    {
      "avgLogprobs": -0.010488096624612808,
      "finishReason": "STOP",
      "content": {
        "role": "model",
        "parts": [
          {
            "functionCall": {
              "args": {
                "$spec": "SO2"
              },
              "name": "molecular-mass"
            }
          },
          {
            "functionCall": {
              "name": "molecular-mass",
              "args": {
                "$spec": "O3"
              }
            }
          },
          {
            "functionCall": {
              "name": "molecu

### Refine the response with functional calls

The following copy of the messages is not required, but it makes repeated experiments easier:

In [38]:
my @messages2 = @messages;

[{parts => [text => What are the masses of SO2, 03, and C2H5OH?
], role => user}]

Let us define an `LLM::Tool` object for each tool:

In [39]:
my @toolObjects = [&molecular-mass, &balance-chemical-equation].map({ LLM::Tool.new($_) });

.say for @toolObjects

Undefined type of parameter ⎡$spec⎦; continue assuming it is a string.


LLMTool(molecular-mass, Convert chemical compound formula (molecule) into molecule mass. * C<$specs | @specs> A string, or a list of strings. * C<$:p> Should the result be a list of pairs or not? (If the first argument is positional, C<@specs>.))
LLMTool(balance-chemical-equation, Balance a chemical equation.)


Make an `LLM::Request` object for each request from the (first) LLM response:

In [40]:
my @requestObjects = $response<candidates>»<content>»<parts>.&flatten»<functionCall>.map({ LLM::ToolRequest.new( $_<name>, $_<args>) });

.say for @requestObjects

LLMToolRequest(molecular-mass, :$spec(SO2), :id(Whatever))
LLMToolRequest(molecular-mass, :$spec(O3), :id(Whatever))
LLMToolRequest(molecular-mass, :$spec(C2H5OH), :id(Whatever))


Using the relevant tool for each request compute tool's response (which are `LLM::ToolResponse` objects):

In [41]:
.say for @requestObjects.map({ generate-llm-tool-response(@toolObjects, $_) })».output

64.058
47.997
46.069


Alternatively, the `LLM::ToolResponse` objects can be converted into hashmaps structured according a particular LLM function calling style (Gemini in this case):

In [42]:
.say for @requestObjects.map({ generate-llm-tool-response(@toolObjects, $_) })».Hash('Gemini')

{functionResponse => {name => molecular-mass, response => {content => 64.058}}}
{functionResponse => {name => molecular-mass, response => {content => 47.997}}}
{functionResponse => {name => molecular-mass, response => {content => 46.069}}}


Process the response:
- Make a request object for each function call request
- Compute the tool results
- Form corresponding user message with those results
- Send the messages to the LLM

In [43]:
my $assistant-message = $response<candidates>[0]<content>;
if $assistant-message<parts> {

    # Find function call parts and make corresponding tool objects
    my @requestObjects;
    for |$assistant-message<parts> -> %part {
        if %part<functionCall> {
            @requestObjects.push: LLM::ToolRequest.new( %part<functionCall><name>, %part<functionCall><args> ) 
        }
    }    

    # Add assistance message
    @messages2.push($assistant-message);

    # Compute tool responses
    my @funcParts = @requestObjects.map({ generate-llm-tool-response(@toolObjects, $_) })».Hash('Gemini');

    # Make and add the user response
    my %function-response =
        role => 'user',
        parts => @funcParts;

    @messages2.push(%function-response);
                
    # Send the second request with function result
    my $final-response = gemini-generate-content(
        @messages2,
        :@tools,
        :$model,
        format => "raku"
    );
                
    say "Assistant: ", $final-response<candidates>[0]<content><parts>».<text>.join("\n");

} else {
    say "Assistant: $assistant-message<content>";
}

Assistant: The masses of SO2, O3, and C2H5OH are 64.058, 47.997, and 46.069 respectively.



**Remark** Compared to the workflows in [AA1, AA2] the code above in simpler, more universal and robust, and handles all tool requests

-----

## Conclusion

We can observe and conclude that LLM function calling workflows are greatly simplified by:

- Leveraging Raku introspection
    - This requires documenting the subs and their parameters.
- Using dedicated classes that represent tool:
    - Definitions, (`LLM::Tool`)
    - Requests, (`LLM::ToolRequest`)
    - Responses, (`LLM::ToolResponse`)
- Having a sub (`generate-llm-tool-response`) that automatically matches request objects to tool objects and produces the corresponding response objects.
    - Note the Gemini's documentation does not show that matching in the corresponding function calling example ["Parallel Function Calling"](https://ai.google.dev/gemini-api/docs/function-calling#parallel_function_calling).

Raku's LLM tools automation is similar to Gemini's ["Automatic Function Calling (Python Only)"](https://ai.google.dev/gemini-api/docs/function-calling?example=weather#automatic_function_calling_python_only).

The Wolfram Language LLM tooling functionalities are reflected in Raku's "LLM::Tooling", [WRI1].

-----

## References

### Articles, blog posts

[AA1] Anton Antonov,
["LLM function calling workflows (Part 1, OpenAI)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["LLM function calling workflows (Part 2, Google's Gemini)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-2-google-gemini/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA3] Anton Antonov,
["LLM function calling workflows (Part 3, Facilitation)"](https://rakuforprediction.wordpress.com/2025/06/08/llm-function-calling-workflows-part-3-facilitation/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[Gem1] Google Gemini,
["Gemini Developer API"](https://ai.google.dev/gemini-api/docs).

[WRI1] Wolfram Research, Inc.
["LLM-Related Functionality" guide](https://reference.wolfram.com/language/guide/LLMFunctions.html).



### Packages 

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::Gemini Raku package](https://github.com/antononcube/Raku-WWW-Gemini),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).